In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime, timedelta

from quantopian.pipeline import Pipeline, CustomFactor
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import QTradableStocksUS, StaticAssets
from quantopian.pipeline.data import morningstar, Fundamentals as F
from quantopian.research import run_pipeline
from quantopian.pipeline.factors import SimpleMovingAverage

class MeanPS(CustomFactor):
    window_length = 252*3 # ~52 weeks * n years
    inputs = [F.ps_ratio]
    
    def compute(self, today, asset_ids, out, ps_ratios):
        ps_mean = np.mean(ps_ratios, axis=0)
        out[:] = ps_mean


# Gets the 52-week high for each asset.
class High(CustomFactor):
    window_length = 252 # ~52 weeks
    inputs = [USEquityPricing.close]
    
    def compute(self, today, asset_ids, out, close_prices):
        out[:] = np.max(close_prices, axis=0)


def make_pipeline():
    """
    A function to create our dynamic stock selector (pipeline). Documentation
    on pipeline can be found here:
    https://www.quantopian.com/help#pipeline-title
    """
    mask = StaticAssets(symbols([
        'AMZN',
        'AYX',
        'BKNG',
        'CLDR',
        'CRM',
        'CRWD',
        'DBX',
        'DDOG',
        'DOCU',
        'EB',
        'ETSY',
        'EXPE',
        'HUBS',
        'LYFT',
        'MDB',
        'MELI',
        'NET',
        'NFLX',
        'NOW',
        'OKTA',
        'PD',
        'RDFN',
        'SE',
        'SHOP',
        'SNAP',
        'SPCE',
        'SPLK',
        'SPOT',
        'SQ',
        'SVNK',
        'TDOC',
        'TEAM',
        'TRIP',
        'TSLA',
        'TWLO',
        'UBER',
        'UPWK',
        'VMW',
        'WDAY',
        'WIX',
        'WORK',
        'YEXT',
        'Z',
        'ZEN',
        'ZM',
        'ZNGA',
        'ZUO'
    ]))
    
    
    #mask &= (F.roe.latest > )
    #mask &= (F.pb_ratio.latest < 1.0)
    
    # rule of 40 := revenue growth (qtr) + income margin > 40
    revenue_growth = F.revenue_growth.latest
    revenue_growth_rank = revenue_growth.rank(ascending=False, mask=mask)
    net_margin = F.net_margin.latest
    rule_40 =  revenue_growth + net_margin
    rule_40_rank = rule_40.rank(ascending=False, mask=mask)  # higher is better
    
    # gross margin
    
    # relative historical self price to sales
    ps_ratio = F.ps_ratio.latest
    ps_ratio_rank = ps_ratio.rank(ascending=True, mask=mask) # lower is better
    mean_ps = MeanPS()
    rel_ps = F.ps_ratio.latest / mean_ps 
    rel_ps_rank = rel_ps.rank(ascending=True, mask=mask)  # lower is better
    
    # run rate := cash on hand / operating expense in quarters
    run_rate = F.cash_and_cash_equivalents.latest / F.operating_expense.latest
    run_rate_rank = run_rate.rank(ascending=False, mask=mask)  # higher is better
    
    # debt to assets
    # R&D % of sales
    # sales & marketing spend % of sales
    # stock compensation
    
    earnings_yield = F.ebit.latest/F.enterprise_value.latest
    roic      = F.roic.latest
    debt_equity = F.total_debt_equity_ratio.latest
    gross_margin = F.gross_margin.latest
    rnd_ocf = F.research_and_development.latest / F.operating_cash_flow.latest
    total_yield = F.total_yield.latest
    
    EY_rank   = earnings_yield.rank(ascending=False, mask=mask)
    roic_rank = roic          .rank(ascending=False, mask=mask)
    debt_equity_rank = debt_equity.rank(ascending=True, mask=mask)  # lower is better
    gross_margin_rank = gross_margin.rank(ascending=False, mask=mask)
    rnd_ocf_rank = rnd_ocf.rank(ascending=False, mask=mask)
    
    # dividend and buy back yield
    total_yield_rank = total_yield.rank(ascending=False, mask=mask)
    
    
    ma50 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=50, mask=mask)
    ma200 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=200, mask=mask)
    mv_yield = ma50 / ma200
    mv_yield_rank = mv_yield.rank(ascending=False, mask=mask)
    
    sector = F.morningstar_sector_code.latest
    
        
    # Factor of yesterday's close price.
    yesterday_close = USEquityPricing.close.latest
    
    market_cap = F.market_cap.latest
    market_cap_rank = market_cap.rank(ascending=False, mask=mask)
    ev = F.enterprise_value.latest
    ev_rank = ev.rank(ascending=False, mask=mask)
    
    debt_assets = F.debtto_assets.latest
    debt_assets_rank = debt_assets.rank(ascending=True, mask=mask)  # lower is better

    current_ratio = F.current_ratio.latest
    current_ratio_rank = current_ratio.rank(ascending=False, mask=mask)  # higher is better
    
    quick_ratio = F.quick_ratio.latest
    quick_ratio_rank = quick_ratio.rank(ascending=False, mask=mask)  # higher is better
    
    high = High()
    
    columns = {
            'close'         : yesterday_close,
            #'sector'        : sector,

            'rule_40'            : rule_40,
            'rule_40_rank'       : rule_40_rank,
            'rel_ps'             : rel_ps,
            'rel_ps_rank'        : rel_ps_rank,
            'run_rate'           : run_rate,
            'run_rate_rank'      : run_rate_rank,
            'gross_margin'  : gross_margin,
            'gross_margin_rank':  gross_margin_rank,
            #'net_margin'         : net_margin,
            'hi52 %'             : yesterday_close / high,
        
            #'earnings_yield': earnings_yield,
            #'ey_rank'       : EY_rank,
            #'roic'          : roic,
            #'roic_rank'     : roic_rank,

            #'debt_equity'   : debt_equity,
            #'debt_equity_rank': debt_equity_rank,
            #'ev'             : ev,
            #'ev_rank'        : ev_rank,
            #'current_ratio'  : current_ratio,
            #'current_ratio_rank' : current_ratio_rank,
            #'quick_ratio'    : quick_ratio,
            #'quick_ratio_rank' : quick_ratio_rank,
            #'debt_assets'    : debt_assets,
            #'debt_assets_rank'  : debt_assets_rank,
        
            'revenue_growth': revenue_growth,
            'revenue_growth_rank': revenue_growth_rank,
            'ps_ratio'      : ps_ratio,
            'ps_ratio_rank' : ps_ratio_rank,
            #'fcf_yield': fcf_yield,
            #'fcfy_rank': FCFY_rank,

            #'ma50'          : ma50,
            #'ma200'         : ma200,  
        }
    
    weights = {
        #'rule_40_rank': 4,
        #'rel_ps_rank': 1,
        'ps_ratio_rank': 1,
        #'run_rate_rank': 1,
        'gross_margin_rank': 1,
        'revenue_growth_rank': 1,
    }
    rank   = np.sum([rank*weights.get(key, 1) for key, rank in columns.items() if key[-4:] == 'rank'])
    rank_col = '_rank'

    display_columns = {key: value for key, value in columns.items()} # if key[-4:] != 'rank'}
    display_columns['_rank'] = rank
    
    pipe = Pipeline(
        columns=display_columns,
        screen=mask
    )
    return pipe

yest = (datetime.now() - timedelta(days=1)).date()
result = run_pipeline(make_pipeline(), yest, yest)\
            .sort_values(by='_rank', ascending=True)\
            .head(30)\
            .sort_index(axis=1)#.dropna()
result